# 📧 Smart Job Application Assistant using Gemini AI

## 🔍 Problem Statement

Job seekers often face challenges in tailoring their resumes and writing personalized emails for every job application. This can reduce their chances of being shortlisted, especially in competitive job markets.

## 🤖 Solution

We developed a **Smart Job Application Assistant** that uses **Generative AI (Gemini 1.5 Pro & Flash)** to:
- Extract structured data from resumes and job descriptions
- Analyze and match skills and experiences
- Generate highly personalized job application emails

This solution automates a tedious and critical part of job hunting, enhancing both efficiency and effectiveness.

---

## 🧠 GenAI Capabilities Demonstrated

This project uses **three powerful capabilities of Generative AI**:

1. **Structured Output / Controlled Generation**
   - The model outputs structured JSON with fields like skills, experience, job title, and summary.

2. **Document Understanding**
   - Gemini analyzes resume and job descriptions from unstructured PDF/text and extracts relevant information.

3. **Long Context Window**
   - Gemini handles large documents with many tokens (e.g., long resumes and multi-paragraph job posts).

These capabilities were chosen as they directly solve real pain points in resume parsing and job matching.


## Install required libraries for Google Generative AI and PDF processing

In [ ]:
!pip install -q google-generativeai PyMuPDF

## Set Up

In [ ]:
import os
import google.generativeai as genai

# Retrieve and configure Google Generative AI with the secret API key
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

flash_model = genai.GenerativeModel('gemini-1.5-flash')
pro_model = genai.GenerativeModel('gemini-1.5-pro')


## 📄Extract Text from Resume (PDF)

We start by extracting the raw text from a resume PDF file using PyMuPDF.


In [ ]:
import fitz  # PyMuPDF
import tempfile
import shutil
import traceback

def extract_text_from_pdf(pdf_file):
    temp_file = None
    try:
        if isinstance(pdf_file, str):
            file_path = pdf_file
        else:
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.pdf')
            temp_file.close()
            with open(temp_file.name, 'wb') as f:
                f.write(pdf_file.read())
            file_path = temp_file.name

        doc = fitz.open(file_path)
        text = ""
        for page_num in range(len(doc)):
            text += doc.load_page(page_num).get_text()
        return text.strip()
    except Exception as e:
        traceback.print_exc()
        return f"Error extracting text from PDF: {str(e)}"
    finally:
        if 'doc' in locals():
            doc.close()
        if temp_file and os.path.exists(temp_file.name):
            os.unlink(temp_file.name)


### Load a sample PDF (upload your own on Kaggle)
Add data → Upload → New Dataset → Select file → Use (/kaggle/input/your-folder-name/your-file.pdf)

In [ ]:
# Load a sample PDF (upload your own on Kaggle)
resume_path = "/kaggle/input/resume/my_resume (2).pdf"  # adjust path
resume_text = extract_text_from_pdf(resume_path)
print(resume_text[:1000])  # Show first few characters


### Enter a sample job description

In [ ]:
job_desc_text = """
We are seeking talented and motivated Full Stack Developer fresher to join our innovative software team. This role offers a valuable opportunity to gain hands-on experience working with cutting-edge technologies including the MERN stack, IoT systems, and time series databases.

Position Overview

As a Full Stack Developer, they will assist in developing and maintaining web applications and IoT solutions. They will work closely with senior developers to learn and contribute to all aspects of our software development lifecycle, from front-end interfaces to back-end systems and database management.

Responsibilities:

Assist in developing web applications using the MERN (MongoDB, Express.js, React, Node.js) stack
Collaborate with senior developers to design, code, and test software solutions for both web and IoT applications
Help implement and maintain REST APIs for seamless integration between front-end and back-end systems.
Assist in setting up and working with time series databases for storing and analyzing sensor data.
Gain experience with pub/sub architecture and MQTT protocol implementation for IoT systems.
Support the development of WebSocket implementations for real-time communication features
Participate in API testing and debugging procedures to ensure application quality.
Learn about IoT system architecture and implementation practices.
Contribute to code reviews and provide constructive feedback.
Keep up-to-date with current programming practices and relevant technologies.

Requirements

Candidates having a degree in Computer Science, Software Engineering, or related field with 0-2 years of experience.
Basic understanding of the MERN stack: MongoDB, Express.js, React, and Node.js
Familiarity with HTML, CSS, JavaScript and Python.
Understanding of REST API principles and architecture.
Interest in IoT technologies and willingness to learn MQTT, pub/sub patterns, and WebSockets.
Basic knowledge of database systems (both SQL and NoSQL).
Familiarity with version control systems like Git.
Eager to learn about time series databases and their applications.
Good problem-solving skills and attention to detail.
Strong desire to learn and adapt to new technologies.
Ability to work Independently with minimum supervision, effectively in a team environment.


Preferred Skills (Not Required but Beneficial) :
Previous experience with any IoT projects or development.
Familiarity with API testing tools and methodologies.
Experience with any pub/sub messaging patterns.
Knowledge of WebSocket implementation.
Exposure to cloud platforms (AWS, Azure, or Google Cloud).

Benefits
Flexible work timings.
Health Insurance.
Paternity Leaves.
"""


## 📧 Generate basic email with Flash 1.5 model

In [ ]:
def generate_basic_email(resume_text, job_description, user_name, company_name, job_role):
    """Generate a basic email using direct approach."""
    prompt = f"""
    You are a professional mail writer.

    Write a personalized email that {user_name} can send to a hiring manager at {company_name} for the role of {job_role}.
    Use the resume content and job description below to tailor the email. The tone should be professional, polite, and confident.

    --- RESUME ---
    {resume_text}

    --- JOB DESCRIPTION ---
    {job_description}

    --- EMAIL FORMAT ---
    Subject: Application for {job_role} at {company_name}

    [Write the full email body here]
    """

    response = flash_model.generate_content(prompt)
    return response.text

In [ ]:
user_name = 'Aritra Paul'
company_name = 'Sample Comp'
job_title = 'Full Stack Developer'

# Call the basic mail generator function
email = generate_basic_email(resume_text, job_desc_text, user_name, company_name, job_title)
print(email)

## 🧠 Analyze resume texts and job description for advanced email generation using gemini pro model
We use Gemini Pro to convert the raw text into structured JSON. This enables automated comparison and email generation.

In [ ]:
import json
import re

def analyze_resume(resume_text):
    """Extract key information from a resume using Gemini."""
    prompt = f"""
    Analyze the following resume and extract key information in a structured JSON format.
    Include the following fields:
    1. name: The candidate's full name
    2. contact_info: Email and phone number if present
    3. summary: A brief professional summary
    4. skills: List of technical and soft skills
    5. experience: List of work experiences with company, title, dates, and achievements
    6. education: Academic background
    7. projects: Any mentioned projects with descriptions
    8. strengths: The candidate's 3-5 main professional strengths based on the resume
    
    Resume:
    {resume_text}
    
    Return only the JSON without any explanations or markdown formatting.
    """
    
    response = pro_model.generate_content(prompt)
    
    # Extract JSON from response
    try:
        json_str = response.text
        # Remove markdown code block formatting if present
        json_str = re.sub(r'```json|```', '', json_str).strip()
        return json.loads(json_str)
    except Exception as e:
        print(f"Error parsing JSON: {str(e)}")
        print(f"Raw response: {response.text}")
        return {"error": "Failed to parse resume data"}
        
def analyze_job_description(job_description):
    """Extract key information from a job description using Gemini."""
    prompt = f"""
    Analyze the following job description and extract key information in a structured JSON format.
    Include the following fields:
    1. company_name: The company posting the job (if mentioned)
    2. job_title: The position title
    3. location: Job location (remote, hybrid, or physical location) if mentioned
    4. job_summary: A brief summary of the role
    5. required_skills: List of required technical and soft skills
    6. preferred_skills: List of preferred or nice-to-have skills
    7. responsibilities: Key job responsibilities
    8. qualifications: Required education and experience
    9. keywords: 5-10 important keywords from the job description
    
    Job Description:
    {job_description}
    
    Return only the JSON without any explanations or markdown formatting.
    """
    
    response = pro_model.generate_content(prompt)
    
    # Extract JSON from response
    try:
        json_str = response.text
        # Remove markdown code block formatting if present
        json_str = re.sub(r'```json|```', '', json_str).strip()
        return json.loads(json_str)
    except Exception as e:
        print(f"Error parsing JSON: {str(e)}")
        print(f"Raw response: {response.text}")
        return {"error": "Failed to parse job description data"}


In [ ]:
resume_data = analyze_resume(resume_text)
job_data = analyze_job_description(job_desc_text)

## ⚒️ Match the skills 

In [ ]:
def match_skills(resume_data, job_data):
    prompt = f"""
    Compare the resume and job description below.
    Output JSON with:
    - matching_skills
    - missing_skills
    - relevant_experience
    - overall_match_percentage
    - strengths_to_highlight
    - suggested_talking_points

    Resume: {json.dumps(resume_data, indent=2)}
    Job: {json.dumps(job_data, indent=2)}
    """
    response = pro_model.generate_content(prompt)
    json_str = re.sub(r'```json|```', '', response.text).strip()
    return json.loads(json_str)


In [ ]:
skills_match = match_skills(resume_data, job_data)
skills_match

## 📧 Generate advanced email with gemini pro model

In [ ]:
def generate_email_advanced(resume_data, job_data, skills_match, recipient_name="Hiring Manager", email_tone="professional", additional_info=""):
    tone_desc = {
        "professional": "formal and polished",
        "enthusiastic": "energetic and passionate",
        "concise": "brief but comprehensive",
        "conversational": "friendly and approachable"
    }.get(email_tone, "professional and clear")

    prompt = f"""
    Write a {tone_desc} job application email to {recipient_name} using:
    Resume: {json.dumps(resume_data, indent=2)}
    Job: {json.dumps(job_data, indent=2)}
    Skills Match: {json.dumps(skills_match, indent=2)}
    Additional Notes: {additional_info}

    Format:
    Subject: [email subject]
    [email body]
    """
    response = pro_model.generate_content(prompt)
    return response.text


In [ ]:
email = generate_email_advanced(resume_data, job_data, skills_match)
print(email)

### ✅ Summary

In this project, we built a **Smart Job Application Assistant** powered by **Gemini Pro** and **Flash**.

---

#### 🔑 Key Features:
- ✅ Structured JSON output from resumes and job descriptions  
- ✅ Skill matching and experience analysis  
- ✅ Personalized, AI-generated email writing using GenAI

---

#### 🧠 GenAI Capabilities Used:
- Structured Output  
- Document Understanding  
- Long Context Window  

---

#### 💡 Future Enhancements:
- 📌 Add vector search for job/resume history  
- 📌 Add evaluation and scoring (GenAI-based evaluation)  
- 📌 Integrate with email automation tools  

---

This project showcases how GenAI can transform a tedious process into an intelligent, automated solution with real-world impact.

> 🚀 Try the **full-featured Gradio-based web app** here:  
🔗 [Smart Job Application Assistant (GitHub)](https://github.com/Aritra212/mail_assistant.git)

here are some screeshots of the application - 
![mail_assistant_1](https://github.com/Aritra212/mail_assistant/raw/main/screenShots/app-img-1.png)

![mail_assistant_2](https://github.com/Aritra212/mail_assistant/raw/main/screenShots/app-img-2.png)

In [ ]:
# (Optional) you can run the following code to save the generated mail into a .txt file
with open("job_application_email.txt", "w") as f:
    f.write(email)